In [29]:
# imports
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import numpy as np
from datetime import datetime

In [17]:
# source url
url = "https://en.wikipedia.org/wiki/List_of_rocket_launch_sites"
# get html data 
data = requests.get(url).text
# parse data 
soup = BeautifulSoup(data, 'html.parser')

In [18]:
html_headers = soup.find('table', class_='wikitable sortable').find_all('th')
headers = []
for h in html_headers:
    headers.append(h.text.strip('\n'))
headers
df = pd.DataFrame(columns=headers)
df.insert(0, "Continent", [])
df['html_link'] = []
df

,Continent,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes,html_link


In [19]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Oceana', 'Launches at Sea']
for i, table in enumerate (soup.find_all('table', class_='wikitable sortable')):
    # print(table.find_all('tr'))
    html_data = table.find_all('tr')
    for col in html_data:
        data = (col.find_all('td'))
        # print (continents[i])
        if data != []:
            country = [data[0].text.strip('\n')]
            location = [data[1].text.strip('\n')]
            try: 
                coordinate = [data[2].find(class_ = 'geo').text]
            except: 
                coordinate = [data[2].find(class_ = 'geo')]
            operational_date = [data[3].text.strip('\n')]
            rocket_launches = [data[4].text.strip('\n')]
            heaviest_rocket = [data[5].text.strip('\n')]
            highest_altitude = [data[6].text.strip('\n')]
            try: 
                notes = data[7]
                notes_text = [data[7].text.strip('\n')]
                link_end = notes.find('a', href = True)['href']
                # if "cite_note" in link_end:
                    # continue
                link_title = link_end.strip('/wiki')
                link = 'https://en.wikipedia.org' + link_end
                html_url = f'<a href="{link}">{link_title} </a>'
            except:
                notes_text = ['']
                html_url = ''
            df_new_row = pd.DataFrame({'Continent': continents[i],
                                    'Country': country, 
                                    'Location': location, 
                                    'Coordinates': coordinate,
                                    'Operational date': operational_date,
                                    'Number of rocket launches': rocket_launches,
                                    'Heaviest rocket launched': heaviest_rocket,
                                    'Highest achieved altitude': highest_altitude,
                                    'Notes': notes_text,
                                    'html_link': html_url})
            df = pd.concat([df, df_new_row])

df

,Continent,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes,html_link
0,Africa,French Algeria,Centre interarmées d'essais d'engins spéciaux ...,31.09951; -2.83581,1947–1967,230,"18,000 kg",Orbital,Operated by France.[1],"<a href=""https://en.wikipedia.org#cite_note-1""..."
0,Africa,Algeria,Reggane,26.71895; 0.27691,1961–1965,10,,,,
0,Africa,Zaire,"Shaba North, Kapani Tonneo OTRAG Launch Center",-7.92587; 28.52766,1975–1979,3,,<50 km,German OTRAG rockets.[2],"<a href=""https://en.wikipedia.org/wiki/OTRAG"">..."
0,Africa,Egypt,Jabal Hamzah ballistic missile test and launch...,30.125750; 30.605139,1962–1973,6,,,Testing SRBMs Al Zafir and Al Kahir[3][4],"<a href=""https://en.wikipedia.org/wiki/SRBM"">S..."
0,Africa,Kenya,"Broglio Space Centre (San Marco), Malindi",-2.94080; 40.21340,1964–1988,27,"20,000 kg",Orbital,"Scout rockets, operated by ASI and Sapienza Un...","<a href=""https://en.wikipedia.org/wiki/Scout_(..."
...,...,...,...,...,...,...,...,...,...,...
0,Oceana,New Zealand,Great Mercury Island,-36.35511; 175.46006,2009,1,60 kg,120 km,Used for the first launch by Rocket Lab of the...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Oceana,New Zealand,"Rocket Lab Launch Complex 1, Mahia Peninsula",-39.26044; 177.86431,2017–,42[89],"13,000 kg[90]",Lunar,Launch site built and operated by Rocket Lab. ...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Launches at Sea,,Ocean Odyssey complex,None,1999–,30,"462,000 kg",Orbital,Mobile satellite launch platform operated by S...,"<a href=""https://en.wikipedia.org/wiki/Sea_Lau..."
0,Launches at Sea,Russia,Russian Delta class submarines,None,1998–,2,"30,000 kg",Orbital,Launch of uncrewed satellites into Earth orbit...,"<a href=""https://en.wikipedia.org/wiki/SLBM"">S..."


In [20]:
df.loc[df['html_link'].str.contains('cite_note', na=False), 'html_link'] = ''
df


,Continent,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes,html_link
0,Africa,French Algeria,Centre interarmées d'essais d'engins spéciaux ...,31.09951; -2.83581,1947–1967,230,"18,000 kg",Orbital,Operated by France.[1],
0,Africa,Algeria,Reggane,26.71895; 0.27691,1961–1965,10,,,,
0,Africa,Zaire,"Shaba North, Kapani Tonneo OTRAG Launch Center",-7.92587; 28.52766,1975–1979,3,,<50 km,German OTRAG rockets.[2],"<a href=""https://en.wikipedia.org/wiki/OTRAG"">..."
0,Africa,Egypt,Jabal Hamzah ballistic missile test and launch...,30.125750; 30.605139,1962–1973,6,,,Testing SRBMs Al Zafir and Al Kahir[3][4],"<a href=""https://en.wikipedia.org/wiki/SRBM"">S..."
0,Africa,Kenya,"Broglio Space Centre (San Marco), Malindi",-2.94080; 40.21340,1964–1988,27,"20,000 kg",Orbital,"Scout rockets, operated by ASI and Sapienza Un...","<a href=""https://en.wikipedia.org/wiki/Scout_(..."
...,...,...,...,...,...,...,...,...,...,...
0,Oceana,New Zealand,Great Mercury Island,-36.35511; 175.46006,2009,1,60 kg,120 km,Used for the first launch by Rocket Lab of the...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Oceana,New Zealand,"Rocket Lab Launch Complex 1, Mahia Peninsula",-39.26044; 177.86431,2017–,42[89],"13,000 kg[90]",Lunar,Launch site built and operated by Rocket Lab. ...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Launches at Sea,,Ocean Odyssey complex,None,1999–,30,"462,000 kg",Orbital,Mobile satellite launch platform operated by S...,"<a href=""https://en.wikipedia.org/wiki/Sea_Lau..."
0,Launches at Sea,Russia,Russian Delta class submarines,None,1998–,2,"30,000 kg",Orbital,Launch of uncrewed satellites into Earth orbit...,"<a href=""https://en.wikipedia.org/wiki/SLBM"">S..."


In [24]:
df = df.dropna(subset=['Coordinates'])
df.tail()

,Continent,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes,html_link
0,Oceana,Marshall Islands,"Reagan Test Site, Omelek Island, Kwajalein Atoll",9.048167; 167.743083,1950s–,,"39,000 kg",Orbital,US-controlled ICBM base converted for satellit...,"<a href=""https://en.wikipedia.org/wiki/Launch_..."
0,Oceana,New Zealand,Birdling's Flat,-43.81700; 172.68300,1980s–,10<,,,,
0,Oceana,New Zealand,Great Mercury Island,-36.35511; 175.46006,2009,1,60 kg,120 km,Used for the first launch by Rocket Lab of the...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Oceana,New Zealand,"Rocket Lab Launch Complex 1, Mahia Peninsula",-39.26044; 177.86431,2017–,42[89],"13,000 kg[90]",Lunar,Launch site built and operated by Rocket Lab. ...,"<a href=""https://en.wikipedia.org/wiki/Rocket_..."
0,Launches at Sea,Denmark,MLP Sputnik,55.04917; 15.60306,2010–,4,"1,630 kg",8.2 km,Mobile satellite launch platform operated by C...,"<a href=""https://en.wikipedia.org/wiki/Copenha..."


for col in html_data:
    data = col.find_all('td')
    if data != []:
        print(data[2].find(class_ = 'geo').text)


In [7]:
df.to_csv('country_data.csv')
df.to_json('country_data.json', orient = 'split')

In [22]:
new_df = pd.DataFrame()

In [30]:
current_year = datetime.now().year
def extract_dates(period):
    # Case 1: year range like '1947–1967' or '1980-2000'
    match = re.match(r'(\d{4})\D+(\d{4})', period)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    # Case 2: decade like '1950s'
    match = re.match(r'(\d{4})s', period)
    if match:
        start = int(match.group(1))
        end = start + 9
        return start, end
    
    # Case 3: single year like '2010'
    match = re.match(r'^\d{4}$', period)
    if match:
        year = int(match.group(0))
        return year, year
    
    match = re.match(r'(\d{4})\D+$', period)
    if match:
        return int(match.group(1)), current_year

    # Unknown format
    return np.nan, np.nan

# Apply the extraction
df['Operational date'].apply(lambda x: pd.Series(extract_dates(x)))

,0,1
0,1947.0,1967.0
0,1961.0,1965.0
0,1975.0,1979.0
0,1962.0,1973.0
0,1964.0,1988.0
...,...,...
0,1950.0,1959.0
0,1980.0,1989.0
0,2009.0,2009.0
0,2017.0,2025.0


In [ ]:
for i, table in enumerate (soup.find_all('table', class_='wikitable sortable')):
    # print(table.find_all('tr'))
    html_data = table.find_all('tr')
    for col in html_data:
        data = (col.find_all('td'))
        # print (continents[i])
        if data != []:
            try: 
                notes = data[7]
                link_end = notes.find('a', href = True)['href']
                if "cite_note" in link_end:
                    continue
                link_title = link_end.strip('/wiki/')
                link = 'en.wikipedia.org' + link_end
                # print(link_title, ': ', link, '\n')
                print(f'<a href="{link}">{link_title}</a>')
            except:
                notes = ['no notes found']

<a href="en.wikipedia.org/wiki/OTRAG">OTRAG text</a>
<a href="en.wikipedia.org/wiki/SRBM">SRBM text</a>
<a href="en.wikipedia.org/wiki/Scout_(rocket_family)">Scout_(rocket_family) text</a>
<a href="en.wikipedia.org/wiki/OTRAG">OTRAG text</a>
<a href="en.wikipedia.org/wiki/Solar_eclipse">Solar_eclipse text</a>
<a href="en.wikipedia.org/wiki/V-2_rocket">V-2_rocket text</a>
<a href="en.wikipedia.org/wiki/V-2">V-2 text</a>
<a href="en.wikipedia.org/wiki/Meteor_(rocket)">Meteor_(rocket) text</a>
<a href="en.wikipedia.org/wiki/V-2">V-2 text</a>
<a href="en.wikipedia.org/wiki/Instituto_Nacional_de_T%C3%A9cnica_Aeroespacial">Instituto_Nacional_de_T%C3%A9cnica_Aeroespacial text</a>
<a href="en.wikipedia.org/wiki/Arcas_(rocket)">Arcas_(rocket) text</a>
<a href="en.wikipedia.org/wiki/European_Space_Research_Organisation">European_Space_Research_Organisation text</a>
<a href="en.wikipedia.org/wiki/Cyclone-4M">Cyclone-4M text</a>
<a href="en.wikipedia.org/wiki/Robert_H._Goddard">Robert_H._Goddard t